<a href="https://colab.research.google.com/github/Lill98/Face-Recognition-using-FaceNet/blob/master/Face_Recognition_using_FaceNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Flow:
1.   Take dataset 
2.   Align_mtcnn
3.   Main
4.   Train & save
5.   Recognition

##1.   Take dataset 

In [ ]:
!pip install mtcnn

In [ ]:
from __future__ import absolute_import,division,print_function

import tensorflow as tf
import numpy as np
import os
import sys
import math
import pickle
from sklearn.svm import SVC
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
def get_dataset(path):
    def get_image_path(file_image_path):
        images_paths=[]
        images=os.listdir(file_image_path)
        num_image=len(images)
        #print("num_image={}".format(num_image))
        for i in range(num_image):
            image_path=os.path.join(file_image_path,images[i])
            #print(image_path)
            
            images_paths.append(image_path)
        return images_paths
    
    class ImageClass():
        def __init__(self,name,image_paths):
            self.name=name
            self.image_paths=image_paths

        def __str__(self):
            return 'list have 2 attribute: name & image_paths. Detail  ' + 'name_label  '+self.name+'  ,  '+'have   '+str(len(self.image_paths))+'   images' 

        def __len__(self):
            return len(self.image_paths)
    
    def get_datasets(path):
        dataset=[]
        path_exp=os.path.expanduser(path)
        classes_label=os.listdir(path_exp)
        print("before sort classer_label  ",classes_label)

        classes_label.sort()
        print("after sort classer_label  ",classes_label)
        
        #logging.debug(classes_label)
        number_classes=len(classes_label)
        for i in range (number_classes):
            file_image_path=os.path.join(path,classes_label[i])
            #print("file_image_path={}".format(file_image_path))
            #logging.debug(file_image_path)
            
            if os.path.isdir(file_image_path):
                image_paths=get_image_path(file_image_path)
                #print("image_paths={}".format(image_paths))
                class_name=classes_label[i]
                #print(type(ImageClass(class_name,image_paths)))
                #print(image_path)
                dataset.append(ImageClass(class_name,image_paths))

        return dataset

    dataset=get_datasets(path)
    return dataset

2.   Align_mtcnn

In [ ]:
def get_output(path,name):
  if not os.path.exists(path):
    os.makedirs(path)
  new_folder=os.path.join(path,name)
  if not os.path.exists(new_folder):
    os.makedirs(new_folder)
  return new_folder

In [ ]:
def align_mtcnn(dataset,output_path):
    import cv2
    import matplotlib.pyplot as plt
    from mtcnn.mtcnn import MTCNN
    from google.colab.patches import cv2_imshow


    detector=MTCNN()
    #plt.figure(figsize=(10,10))
    for i in range(len(dataset)):
    #plt.subplot(1,2,i+1)
        image_path=dataset[i].image_paths[0]
        image_name=dataset[i].name
        logging.debug(image_name)
        file_image_path=get_output(output_path,image_name)
        image=cv2.imread(image_path)
        image=cv2.resize(image,(250,250))
        result=detector.detect_faces(image)
        for person in result:
            bounding_box=person["box"]
            start_y=person["box"][1]
            end_y=person["box"][3]+start_y
            start_x=person["box"][0]
            end_x=person["box"][2]+start_x
            image_crop=image[start_y:end_y,start_x:end_x,:]
            _,name_image=os.path.split(image_path)
            logging.debug(name_image)
            path_image_out=os.path.join(file_image_path,name_image)
            if not os.path.exists(path_image_out):
              cv2.imwrite(path_image_out,image_crop)
            
            cv2_imshow(image_crop)
            print(image_crop.shape)

3.   Main

lấy emb_array vs labels của ảnh

In [ ]:
def main(input_path,dataset,model_path):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            def get_image_paths_and_labels(dataset):
                image_paths_flat=[]
                labels_flat=[]
                for i in range(len(dataset)):
                    image_paths_flat+=dataset[i].image_paths
                    labels_flat+=[i]*len(dataset[i].image_paths)
                return image_paths_flat,labels_flat
            dataset=get_dataset(input_path)

            paths,labels=get_image_paths_and_labels(dataset)
            print("paths:   {} \n labels:   {}".format(paths,labels))

            from tensorflow.python.platform import gfile
            def load_model(model, input_map=None):
                # Check if the model is a model directory (containing a metagraph and a checkpoint file)
                #  or if it is a protobuf file with a frozen graph
                model_exp = os.path.expanduser(model)
                if (os.path.isfile(model_exp)):
                    print('Model filename: %s' % model_exp)
                    with gfile.FastGFile(model_exp,'rb') as f:
                        graph_def = tf.GraphDef()
                        graph_def.ParseFromString(f.read())
                        tf.import_graph_def(graph_def, input_map=input_map, name='')
                else:
                    print('Model directory: %s' % model_exp)
                    meta_file, ckpt_file = get_model_filenames(model_exp)
                    
                    print('Metagraph file: %s' % meta_file)
                    print('Checkpoint file: %s' % ckpt_file)
                
                    saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file), input_map=input_map)
                    saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))

            load_model(model_path)

            images_placeholder=tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings=tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder=tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size=embeddings.get_shape()[1]
            
            def to_rgb(img):
                w,h=img.shape
                ret=np.empty((w,h,3),dtype=np.uint8)
                ret[:,:,0]=ret[:,:,1]=ret[:,:,2]=img
                return ret
            def prewhiten( x):
                mean = np.mean(x)
                std = np.std(x)
                std_adj = np.maximum(std, 1.0/np.sqrt(x.size))
                y = np.multiply(np.subtract(x, mean), 1/std_adj)
                return y  
            from PIL import Image
            def load_data(path,image_size):
                nrof_samples=len(path)
                print("len(nrof_sample)=   ",nrof_samples)
                images=np.zeros((nrof_samples,image_size,image_size,3))
                for i in range(nrof_samples):
                    img=cv2.imread(path[i])
                    img=cv2.resize(img,(160,160,3))
                    #img=np.array(Image.open(path[i]))
                    #img=np.resize(img,(160,160,3))
                    #print("image before standard deviation",img)
                    img=prewhiten(img)
                    #print("image after  standard deviation",img)
                    #print("kich thuoc cua anh {}".format(img.shape))
                    if(img.ndim==2):
                        img=to_rgb(img)
                    images[i,:,:,:]=img
                    #print("img =  {}".format(img))
                    #print("images[0,:,:,:]=   {}".format(images[0,:,:,:]))
                return images
            
            batch_size=1
            image_size=160
            nrof_images=len(paths)
            nrof_batches_per_epoch=int(math.ceil(1.0*nrof_images/batch_size))
            emb_array=np.zeros((nrof_images,embedding_size))
            #print(emb_array)
            for i in range(nrof_batches_per_epoch):
                start_index=i*batch_size
                end_index=min((i+1)*batch_size,nrof_images)
                paths_batch=paths[start_index:end_index]
                images=load_data(paths_batch,image_size)
                
                #print("images_i=   {}".format(images))
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
                #print("emb_array_i=  {}".format(emb_array))
            #print("full_emb_array=  {}".format(emb_array))
            
            class_names=[cls.name.replace('_',' ')for cls in dataset]
            return (emb_array,class_names)

4.   Train & save

In [ ]:
def save_distance(dataset,output_path,emb_train,name_output_directory):
    if not os.path.exists(name_output_directory):
      os.makedirs(name_output_directory)

    class_names=[cls.name.replace('_',' ')for cls in dataset]
    with open(output_path,'wb') as outfile:
        pickle.dump((emb_train,class_names),outfile)


5.   Recognition: Caculate distance between Data and input image

In [ ]:
def calculate_recognize(emb_array_test,trained_emb_test_path):
    # print(emb_array_test.shape)
    # print(emb_array_train.shape)

    with open(trained_emb_test_path,'rb') as infile:
      (emb_array_train,class_names)=pickle.load(infile)
    print("(class_names=  :{})".format(class_names))
    print("leng emb_array=",len(emb_array_train))
    print("finished load classifier model")
    # for h in range(len(emb_array_test)):
    #print("nguoi thu {}".format(h))

    for k in range(len(emb_array_test)):
        dist=[]
        sub_vect=[]
        for i in range(len(emb_array_train)):
            sub_vect.append((emb_array_test[k]-emb_array_train[i]))
            # print(len(sub_vect[0][0]))
            # print(sub_vect[0][0][0])
            # print(len(sub_vect[0]))

        #print(sub_vect.shape)
        for i in range(len(sub_vect)):
            s=0
            for j in range(len(sub_vect[i])):
                # print(sub_vect[i][0][])
                s=s+sub_vect[i][j]*sub_vect[i][j]
            s=np.sqrt(s)
            dist.append(s)
        print(dist)
        min=dist[0]   
        j=0 
        for i in range(len(dist)):
            if dist[i]<min:
                min=dist[i]
                j=i
        if min<1:
            print("anh duoc nhan dang la cua   {}".format(class_names[j]))

Test program

In [ ]:
path_train="put your path here"
out_put_mtcnn="put your path here"
model_path="put your path here"
output_data_dir="put your path here"
output_data_path="put your path here"

path_test="put your path here"
out_put_mtcnn_test="put your path here"

In [ ]:
#process for train_Data
dataset_train=get_dataset(path_train)
align_mtcnn(dataset_train,out_put_mtcnn)
emb_array_train,labels=main(input_path=out_put_mtcnn,dataset=dataset_train,model_path=model_path)
save_distance(dataset= dataset_train,output_path=output_data_path,emb_train= emb_array_train,name_output_directory=output_data_dir)

In [ ]:
#process for train_Data
output_path=output_data_path
dataset_test=get_dataset(path_test)
print(dataset_test[0].image_paths)
align_mtcnn(dataset_test,out_put_mtcnn_test)
emb_array_test,labels=main(input_path=out_put_mtcnn_test,dataset=dataset_test,model_path=model_path)


In [ ]:
#recognize
nrof_images=2
nrof_flips=3;
image_paths=4
labels_array = np.expand_dims(np.arange(0,nrof_images),1)
print(labels_array)
print(np.array(image_paths))
print(np.repeat(np.array(image_paths),nrof_flips))
image_paths_array = np.expand_dims(np.repeat(np.array(image_paths),nrof_flips),1)
print("image_paths_array",image_paths_array)

control_array = np.zeros_like(labels_array, np.int32)
print("control_array",control_array)
print(4/2)

In [ ]:
print("label name of test  {}".format(labels))
calculate_recognize(emb_array_test,output_data_path)